In [ ]:
# The data augmentaiton function defined in this block are only used for the synthetic data

import tensorflow as tf
from tensorflow.keras import backend as K

def random_t_shifts(d):
   for i in range(d.shape[1]):
     d[:,i] = np.roll(d[:,i], np.random.choice(1024))
   return d

def full_field_noise(data):

  #files = glob.glob(cfg.PATH_FIELD_NOISE+'/*')
  #fn = random.choice(files)
  #noise = np.load(fn)

  # pick random time-window
  noise_idx = random.choice(np.arange(cfg.field_noise.shape[0]))
  idx = random.choice(np.arange(0, cfg.field_noise.shape[1]-cfg.N_TIMESAMPLES))
  noise = cfg.field_noise[noise_idx, idx:idx+cfg.N_TIMESAMPLES, :]
  #idx = random.choice(np.arange(0,noise.shape[0]-cfg.N_TIMESAMPLES))
  #noise = cfg.field_noise[noise_idx, idx:idx+cfg.N_TIMESAMPLES, :]

  #noise = noise[idx:idx+cfg.N_TIMESAMPLES, :]
  #noise = normalize(noise)
  noise = np.random.uniform(1.0,8.0)*normalize(noise)
  #if random.choice([True, False]) == True:
  #  noise = np.flipud(noise)
  #if random.choice([True, False]) == True:
  #  noise = np.random.permutation(noise.T).T
  #if random.choice([True, False]) == True:
  #  noise = random_t_shifts(noise)
  
  #noise = np.reshape(noise, (noise.shape[0], noise.shape[1], 1))
  data = normalize(data)+noise

  return data

def tf_full_field_noise(data: tf.Tensor) -> tf.Tensor:
  data_shape = data.shape
  [data,] = tf.py_function(full_field_noise, [data], [tf.float32])
  data.set_shape(data_shape)
  return data

def station_dropout(data):
    n_drop = random.randrange(cfg.MIN_DROP, cfg.MAX_DROP, 1)
    drop_stations = np.random.choice(np.arange(data.shape[1]), n_drop, replace=False)
    mask = np.ones((data.shape))
    mask[:, drop_stations] *= 0
    data = np.multiply(data, mask)
    return data

def tf_station_dropout(data: tf.Tensor) -> tf.Tensor:
    data_shape = data.shape
    [data,] = tf.py_function(station_dropout, [data], [tf.float32])
    data.set_shape(data_shape)
    return data

def time_shift(data, z):
    negative_time_shifts = np.array([-100, -150, -200, -240, -270, -310])*0.7 #np.array([-20, -30, -40, -48, -54, -62]) #np.array([-100, -150, -200, -240, -270, -310])
    positive_time_shifts = np.array([550, 500, 450, 400, 350, 300])*0.7 #np.array([120, 112, 104, 96, 88, 80]) #np.array([600, 560, 520, 480, 440, 400])
    
    ind_label = np.argmax(z)
    t_shift = np.random.randint(negative_time_shifts[ind_label], positive_time_shifts[ind_label])
  
    data = np.roll(data, t_shift, axis=0)

    if t_shift < 0:
        data[t_shift:, :] = 0
    elif t_shift > 0:
        data[:t_shift, :] = 0

    return data

def tf_time_shift(data, z):
    data_shape = data.shape
    [data,] = tf.py_function(time_shift, [data, z], [tf.float32])
    data.set_shape(data_shape)
    return data

def gaussian_noise(data):
    noise_stations = np.random.choice(cfg.GAUSS_NOISE.shape[1], 96, replace=False)
    random_noise = cfg.GAUSS_NOISE[:,noise_stations]
    t_shift = np.random.randint(0,96)
    random_noise = np.roll(random_noise, t_shift, axis=1)
    t_shift = np.random.randint(0,1024)
    random_noise = np.roll(random_noise, t_shift, axis=0)
    random_noise = np.random.uniform(0.01, 0.2, 96)*random_noise
    data = data/np.max(np.abs(data))
    data = data+ random_noise
    data = data/np.max(np.abs(data))
    return data

def tf_gaussian_noise(data):
    data_shape = data.shape
    [data,] = tf.py_function(gaussian_noise, [data], [tf.float32])
    data.set_shape(data_shape)
    return data

def normalize(data):
    data = data/np.max(np.abs(data))
    return data

def tf_normalize(data):
    return tf.math.divide(data, tf.math.reduce_max(tf.math.abs(data)))

def FCN_output(sx):
    x = np.reshape(np.linspace(5500,8500,128), (128,1,1)) #np.reshape(np.linspace(5300,8700,88), (88,1,1))
    y = np.reshape(np.linspace(3500,6100,96), (1,96,1)) #np.reshape(np.linspace(3300,6300,48), (1,48,1
    z = np.reshape(np.linspace(1400,3600,64), (1,1,64)) # np.reshape(np.linspace(1200,3800,32), (1,1,32)
    xc = np.round(sx[0])
    yc = np.round(sx[1])
    zc = np.round(sx[2])
    fcn_out = np.exp(-((x-xc)**2/(2*200**2)+(y-yc)**2/(2*200**2)+(z-zc)**2/(2*200**2)))#np.exp(-((x-xc)**2+(y-yc)**2+(z-zc)**2)/(2*300**2))#np.exp(-((x-xc)**2/(2*200**2)+(y-yc)**2/(2*200**2)+(z-zc)**2/(2*200**2)))
    fcn_out = fcn_out/np.max(fcn_out)
    return fcn_out

def tf_FCN_output(xc,yc,zc):
    [fcn,] = tf.py_function(FCN_output, [[xc,yc,zc]], [tf.float32])
    fcn.set_shape((128,96,64))
    return fcn

def FCN_output_perturbed(sx):
    x = np.reshape(np.linspace(5500,8500,128), (128,1,1)) #np.reshape(np.linspace(5300,8700,88), (88,1,1))
    y = np.reshape(np.linspace(3500,6100,96), (1,96,1)) #np.reshape(np.linspace(3300,6300,48), (1,48,1
    z = np.reshape(np.linspace(1400,3600,64), (1,1,64)) # np.reshape(np.linspace(1200,3800,32), (1,1,32)

    loc_errors = [sx[3], sx[4], sx[5]]

    rx = np.random.normal(loc=sx[0], scale=np.std(np.arange(-int(loc_errors[0]),int(loc_errors[0])+1)))
    ry = np.random.normal(loc=sx[1], scale=np.std(np.arange(-int(loc_errors[1]),int(loc_errors[1])+1)))
    rz = np.random.normal(loc=sx[2], scale=np.std(np.arange(-int(loc_errors[2]),int(loc_errors[2])+1)))

    while abs(rx-sx[0]) > loc_errors[0]:
      rx = np.random.normal(loc=sx[0], scale=np.std(np.arange(-int(loc_errors[0]),int(loc_errors[0])+1)))
    while abs(ry-sx[1]) > loc_errors[1]:
      ry = np.random.normal(loc=sx[1], scale=np.std(np.arange(-int(loc_errors[1]),int(loc_errors[1])+1)))
    while abs(rz-sx[2]) > loc_errors[2]:
      rz = np.random.normal(loc=sx[2], scale=np.std(np.arange(-int(loc_errors[2]),int(loc_errors[2])+1)))

    xc = rx
    yc = ry
    zc = rz

    fcn_out = np.exp(-((x-xc)**2/(2*200**2)+(y-yc)**2/(2*200**2)+(z-zc)**2/(2*200**2)))#np.exp(-((x-xc)**2+(y-yc)**2+(z-zc)**2)/(2*300**2))#np.exp(-((x-xc)**2/(2*200**2)+(y-yc)**2/(2*200**2)+(z-zc)**2/(2*200**2)))
    fcn_out = fcn_out/np.max(fcn_out)
    return fcn_out

def tf_FCN_output_perturbed(xc,yc,zc, rx, ry, rz):
    [fcn,] = tf.py_function(FCN_output, [[xc,yc,zc, rx, ry, rz]], [tf.float32])
    fcn.set_shape((128,96,64))
    return fcn

In [ ]:
import tensorflow as tf
import random
import os

# Create an input function reading a file using the Dataset API
def read_dataset(prefix, batch_size, cfg):
  def _input_fn(example_serialized):
    feature_map = {
        "x_true": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "y_true": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "z_true": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "sx_error": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "sy_error": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "sz_error": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "z_id": tf.io.FixedLenFeature(shape=[6], dtype=tf.float32),
        "data": tf.io.FixedLenFeature(shape=[cfg.N_TIMESAMPLES, cfg.N_TRACES], dtype=tf.float32),
        }

    parsed = tf.io.parse_single_example(example_serialized, feature_map)
    data = parsed["data"]

    if tf.math.reduce_max(parsed["z_id"])>=0:
      data = tf_time_shift(data, parsed["z_id"])
      #data = tf_gaussian_noise(data)
      data = tf_full_field_noise(data)
      data = tf_station_dropout(data)
      label = tf_FCN_output(parsed["x_true"], parsed["y_true"], parsed["z_true"])
    else:
      label = tf_FCN_output(parsed["x_true"], parsed["y_true"], parsed["z_true"])
      #label = tf_FCN_output_perturbed(parsed["x_true"], parsed["y_true"], parsed["z_true"],
      #                                parsed["sx_error"], parsed["sy_error"], parsed["sz_error"])

    data = tf_normalize(data)
    data = tf.reshape(data, (cfg.N_TIMESAMPLES, cfg.N_TRACES, 1))
    
    return (data, label)

  def _input_fn2(example_serialized):
    feature_map = {
        "x_true": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "y_true": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "z_true": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "z_id": tf.io.FixedLenFeature(shape=[6], dtype=tf.float32),
        "data": tf.io.FixedLenFeature(shape=[cfg.N_TIMESAMPLES, cfg.N_TRACES], dtype=tf.float32),
        }

    parsed = tf.io.parse_single_example(example_serialized, feature_map)
    data = parsed["data"]

    label = tf_FCN_output(parsed["x_true"], parsed["y_true"], parsed["z_true"])

    data = tf_normalize(data)
    data = tf.reshape(data, (cfg.N_TIMESAMPLES, cfg.N_TRACES, 1))
    
    return (data, label)

  if prefix=="train":

    # Get synthetic and field data
    file_path_synth = os.path.join(cfg.PATH_TO_SYNTH_DATA,'%s*' % prefix)
    file_path_field = os.path.join(cfg.PATH_TO_FIELD_DATA, '%s*' % prefix)

    # Create list of files that match pattern and process synthetic dataset
    file_list_synth = tf.io.matching_files(file_path_synth)
    file_list_synth = tf.random.shuffle(file_list_synth)
    file_list_field = tf.io.matching_files(file_path_field)
    if cfg.TRANSFER_LEARNING == True:
      #file_list_synth = tf.random.shuffle(file_list_synth)
      file_list_synth = tf.slice(file_list_synth, [0], [cfg.N_SYNTH_FILES])
      file_list = tf.concat([file_list_synth, file_list_field], axis=-1) # combine field and synthetic data list
      shards = tf.data.Dataset.from_tensor_slices(file_list)
      shards = shards.shuffle(tf.cast(tf.shape(file_list)[0], tf.int64))
      shards = shards.repeat()
      #dataset = dataset.map(_input_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
      #ataset = dataset.repeat(num_epochs).batch(batch_size)
    else:
      file_list = tf.concat([file_list_synth, file_list_field], axis=-1)
      shards = tf.data.Dataset.from_tensor_slices(file_list)
      shards = shards.shuffle(tf.cast(tf.shape(file_list)[0], tf.int64))
      shards = shards.repeat()
    # Feed the shards into TFRecordDataset and randomize again with interleave.
    dataset = shards.interleave(tf.data.TFRecordDataset, cycle_length=4, block_length=16, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=10*batch_size)
    num_epochs=None
    dataset = dataset.map(_input_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.repeat(num_epochs).batch(batch_size)


  if prefix == "val":
    file_path_field = os.path.join(cfg.PATH_TO_FIELD_DATA, '%s*' % prefix)
    file_list_field = tf.io.matching_files(file_path_field)
    shards = tf.data.Dataset.from_tensor_slices(file_list_field)
    dataset = tf.data.TFRecordDataset(shards)
    num_epochs = 1
    dataset = dataset.map(_input_fn2, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.repeat(num_epochs).batch(batch_size)

  return dataset.prefetch(tf.data.experimental.AUTOTUNE)


In [ ]:
import tensorflow as tf
import random
import os

# Create an input function reading a file using the Dataset API
def read_dataset_field(prefix, batch_size, cfg):
  def _input_fn(example_serialized):
    feature_map = {
        "x_true": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "y_true": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "z_true": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "sx_error": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "sy_error": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "sz_error": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "z_id": tf.io.FixedLenFeature(shape=[6], dtype=tf.float32),
        "data": tf.io.FixedLenFeature(shape=[cfg.N_TIMESAMPLES, cfg.N_TRACES], dtype=tf.float32),
        }

    parsed = tf.io.parse_single_example(example_serialized, feature_map)
    data = parsed["data"]

    label = tf_FCN_output(parsed["x_true"], parsed["y_true"], parsed["z_true"])
    data = tf_normalize(data)
    data = tf.reshape(data, (cfg.N_TIMESAMPLES, cfg.N_TRACES, 1))
    
    return (data, label)

  if prefix=="train":

    # Get synthetic and field data
    file_path_field = os.path.join(cfg.PATH_TO_FIELD_DATA, '%s*' % prefix)

    # Create list of files that match pattern and process synthetic dataset
    file_list_field = tf.io.matching_files(file_path_field)
    if cfg.TRANSFER_LEARNING == True:
      shards = tf.data.Dataset.from_tensor_slices(file_list_field)
      shards = shards.shuffle(tf.cast(tf.shape(file_list_field)[0], tf.int64))
      shards = shards.repeat()
    else:
      shards = tf.data.Dataset.from_tensor_slices(file_list_field)
      shards = shards.shuffle(tf.cast(tf.shape(file_list)[0], tf.int64))
      shards = shards.repeat()
    # Feed the shards into TFRecordDataset and randomize again with interleave.
    dataset = shards.interleave(tf.data.TFRecordDataset, cycle_length=4, block_length=16, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=10*batch_size)
    num_epochs=None
    dataset = dataset.map(_input_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.repeat(num_epochs).batch(batch_size)


  if prefix == "val":
    file_path_field = os.path.join(cfg.PATH_TO_FIELD_DATA, '%s*' % prefix)
    file_list_field = tf.io.matching_files(file_path_field)
    shards = tf.data.Dataset.from_tensor_slices(file_list_field)
    dataset = tf.data.TFRecordDataset(shards)
    num_epochs = 1
    dataset = dataset.map(_input_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.repeat(num_epochs).batch(batch_size)

  return dataset.prefetch(tf.data.experimental.AUTOTUNE)


In [ ]:
import tensorflow as tf
import random
import os

# Create an input function reading a file using the Dataset API
def read_dataset_synth(prefix, batch_size, cfg):
  def _input_fn(example_serialized):
    feature_map = {
        "x_true": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "y_true": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "z_true": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "sx_error": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "sy_error": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "sz_error": tf.io.FixedLenFeature(shape=[], dtype=tf.float32, default_value=-1),
        "z_id": tf.io.FixedLenFeature(shape=[6], dtype=tf.float32),
        "data": tf.io.FixedLenFeature(shape=[cfg.N_TIMESAMPLES, cfg.N_TRACES], dtype=tf.float32),
        }

    parsed = tf.io.parse_single_example(example_serialized, feature_map)
    data = parsed["data"]

    data = tf_time_shift(data, parsed["z_id"])
    data = tf_full_field_noise(data)
    data = tf_station_dropout(data)
    label = tf_FCN_output(parsed["x_true"], parsed["y_true"], parsed["z_true"])
    data = tf_normalize(data)
    data = tf.reshape(data, (cfg.N_TIMESAMPLES, cfg.N_TRACES, 1))
    
    return (data, label)

  if prefix=="train":

    # Get synthetic and field data
    file_path_synth = os.path.join(cfg.PATH_TO_SYNTH_DATA,'%s*' % prefix)

    # Create list of files that match pattern and process synthetic dataset
    file_list_synth = tf.io.matching_files(file_path_synth)
    file_list_synth = tf.random.shuffle(file_list_synth)
    if cfg.TRANSFER_LEARNING == True:
      file_list_synth = tf.slice(file_list_synth, [0], [cfg.N_SYNTH_FILES])
      shards = tf.data.Dataset.from_tensor_slices(file_list_synth)
      shards = shards.shuffle(tf.cast(tf.shape(file_list_synth)[0], tf.int64))
      shards = shards.repeat()
    else:
      shards = tf.data.Dataset.from_tensor_slices(file_list_synth)
      shards = shards.shuffle(tf.cast(tf.shape(file_list)[0], tf.int64))
      shards = shards.repeat()
    # Feed the shards into TFRecordDataset and randomize again with interleave.
    dataset = shards.interleave(tf.data.TFRecordDataset, cycle_length=4, block_length=16, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=10*batch_size)
    num_epochs=None
    dataset = dataset.map(_input_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.repeat(num_epochs).batch(batch_size)


  if prefix == "val":
    file_path_field = os.path.join(cfg.PATH_TO_FIELD_DATA, '%s*' % prefix)
    file_list_field = tf.io.matching_files(file_path_field)
    shards = tf.data.Dataset.from_tensor_slices(file_list_field)
    dataset = tf.data.TFRecordDataset(shards)
    num_epochs = 1
    dataset = dataset.map(_input_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.repeat(num_epochs).batch(batch_size)

  return dataset.prefetch(tf.data.experimental.AUTOTUNE)


In [ ]:
def dice_coef_gaussian(y_true, y_pred, smooth=1):
  y_true = tf.dtypes.cast(y_true>0.1, tf.int32)
  y_true = tf.dtypes.cast(y_true, tf.float32)
  y_pred = tf.dtypes.cast(y_pred>0.1, tf.int32)
  y_pred = tf.dtypes.cast(y_pred, tf.float32)
  intersection = K.sum(y_true * y_pred, axis=[1,2,3])
  union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
  dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
  return dice

def dice_coef(y_true, y_pred, smooth=1):
  y_true = tf.dtypes.cast(y_true, tf.float32)
  y_pred = tf.dtypes.cast(y_pred, tf.float32)
  intersection = K.sum(y_true * y_pred, axis=[1,2,3])
  union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
  dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
  return dice

# Clip values in Gaussian above 0.1 to 1 and below 0.1 to 0
def iou_coef_gaussian(y_true, y_pred, smooth=1):
    y_true = tf.dtypes.cast(y_true>0.1, tf.int32)
    y_true = tf.dtypes.cast(y_true, tf.float32)
    y_pred = tf.dtypes.cast(y_pred>0.1, tf.int32)
    y_pred = tf.dtypes.cast(y_pred, tf.float32)
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def train_and_evaluate(cfg):


  checkpoint_filepath = 'drive/My Drive/Texas_TL_sourceLocalization/checkpoint'
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      save_weights_only=True,
      monitor='val_dice_coef',
      mode='max',
      save_best_only=True
  )
  print("loading model...")
  generator = tf.keras.models.load_model(
      os.path.join(cfg.PATH_TO_MODELS, cfg.LOAD_MODEL),
      custom_objects={
          'ReLU':tf.keras.layers.ReLU,
          'dice_coef_gaussian': dice_coef_gaussian, 'iou_coef_gaussian': iou_coef_gaussian,
          })
  
  generator.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=cfg.LEARNING_RATE),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[dice_coef_gaussian, dice_coef])#metrics=['accuracy', iou_coef_gaussian, dice_coef_gaussian]

  # Compute train and validation steps
  file_path = os.path.join(cfg.PATH_TO_SYNTH_DATA,'%s*' % cfg.TRAIN)
  file_list = tf.io.matching_files(file_path)

  file_list = file_list[:cfg.N_SYNTH_FILES]
  n_train_examples_synth = len(file_list)*cfg.N_FILES_SYNTH_TFRECORD 
  n_train_files_field = cfg.N_FILES_FIELD_TFRECORD_TRAIN
  n_train_examples = n_train_files_field + n_train_examples_synth

  steps_per_epoch = n_train_examples // cfg.BATCH_SIZE
  val_steps = cfg.N_FILES_FIELD_TFRECORD_VAL // cfg.BATCH_SIZE
  
  for i in range(100):
    # Load the training data
    print(i)
    trainds = read_dataset(prefix=cfg.TRAIN, batch_size=cfg.BATCH_SIZE, cfg=cfg)
    valds = read_dataset(prefix=cfg.VAL, batch_size=cfg.BATCH_SIZE, cfg=cfg)
    generator.fit(
        trainds,
        validation_data=valds,
        epochs=cfg.N_EPOCHS,
        steps_per_epoch=steps_per_epoch,
        validation_steps=val_steps,
        verbose=1,
        callbacks=[model_checkpoint_callback])
    
    save_as = os.path.join(cfg.PATH_TO_TL_MODELS, "{}_{}.h5".format(cfg.MODEL_NAME, i))
    generator.save(save_as, overwrite=True, include_optimizer=True)

In [ ]:
import glob
import numpy as np

files = glob.glob("drive/My Drive/Texas_TL/Data/contNoise_newShape/*")

#field_noise = np.load(files[0])
#n_size = field_noise.shape
#field_noise = np.reshape(field_noise, (1, n_size[0], n_size[1]))
#for i, fn in enumerate(files[1:]):
#  field_noise = np.append(field_noise, np.reshape(np.load(fn), (1,n_size[0], n_size[1])), axis=0)

class Config(object):
  def __init__(self):
    # PATHS TO MODELS AND OUTPUT
    self.PATH_TO_SYNTH_DATA = "drive/My Drive/Texas_TL/Data/TFR_new"
    self.PATH_TO_FIELD_DATA = "drive/My Drive/Texas_TL/Data/TFR_field"#_days4"
    self.PATH_TO_MODELS = "drive/My Drive/Texas_TL/Models/original_QNet"
    self.PATH_FIELD_NOISE = "drive/My Drive/Texas_TL/Data/contNoise_newShape"
    self.PATH_TO_TL_MODELS = "drive/My Drive/Texas_TL/TL_models/TL_TFR_regular_updating"
    # DEFINE SIZE OF INPUT, BATCH SIZE, NUMBER OF EPOCHS (EVALS), LEARNING RATE, OTHER FIXED NUMBERS
    self.BATCH_SIZE = 10
    self.N_TRACES = 96
    self.N_TIMESAMPLES = 1024
    self.N_EPOCHS = 4
    self.LEARNING_RATE =  0.001
    self.N_FILES_SYNTH_TFRECORD = 400
    self.N_FILES_FIELD_TFRECORD_TRAIN = 747 #747 for all train set # 357  #477 for combo_train_val
    self.N_FILES_FIELD_TFRECORD_VAL = 249 # 249 #  120
    self.MIN_DROP = 5
    self.MAX_DROP = 25
    # NAME FOR SAVING THE MODEL
    self.MODEL_NAME = "QNet_post_test_long" #"QNet-Synth2000-noPrtbn-"
    # PREFIX FOR TRAINING AND VALIDATION SETS
    self.TRAIN = "train"
    self.VAL = "val"
    # LOAD FIELD NOISE, SINGLE FIELD NOISE AND GAUSSIAN NOISE
    self.field_noise = field_noise
    # NAME FOR CHECKPOINTS
    self.LOAD_MODEL = "QNet_FieldGaus8t_20ep_20210324182729.h5" # name of model to load before starting training
    self.TRANSFER_LEARNING = True
    self.N_SYNTH_FILES = 0

In [ ]:
cfg = Config()
train_and_evaluate(cfg)